In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import os

# === 1. Cargar simulaciones ===
ruta_csv_pv = 'resumen_simulaciones_pvwatts_metrico.csv'
ruta_csv_csp = 'resultados_simulacion_csp_con_almacenamiento.csv'
df_pv = pd.read_csv(ruta_csv_pv)
df_csp = pd.read_csv(ruta_csv_csp)

# === 2. Definir parámetros de análisis de sensibilidad ===
tasas_descuento = [0.06, 0.08, 0.10, 0.12]
vidas_utiles = [20, 25, 30]

opex_porcentajes_pv = [0.01, 0.02, 0.03]
eficiencias_inversor = [0.94, 0.96, 0.98]
capex_values_pv = [800, 1000, 1200]
degradaciones = [0.03, 0.05]
perdidas_sistema = [0.10, 0.12]

opex_porcentajes_csp = [0.02, 0.03, 0.04]
eficiencias_termicas = [0.35, 0.40, 0.45]
capex_values_csp = [3000, 4000, 5000]
eficiencias_opticas = [0.70, 0.75, 0.80]
factores_perdidas = [0.10, 0.15, 0.20]

# === 3. Funciones para LCOE ===
def produccion_total_degradada(produccion_inicial, degradacion, vida_util):
    if degradacion == 0:
        return produccion_inicial * vida_util
    return produccion_inicial * (1 - degradacion ** vida_util) / (vida_util * (1 - degradacion))

def calcular_lcoe_pv(capacidad, produccion_anual, capex, opex_pct, vida_util, tasa_descuento, degradacion, perdidas, eficiencia_inv):
    produccion_neta = produccion_anual * eficiencia_inv * (1 - perdidas)
    produccion_media_anual = produccion_total_degradada(produccion_neta, degradacion, vida_util)
    costo_total = capacidad * capex
    opex_anual = costo_total * opex_pct
    fcr = (tasa_descuento * (1 + tasa_descuento) ** vida_util) / ((1 + tasa_descuento) ** vida_util - 1)
    costo_anual = costo_total * fcr + opex_anual
    lcoe = costo_anual / produccion_media_anual
    return lcoe, produccion_neta

def calcular_lcoe_csp(area_colectora, produccion_anual, capex, opex_pct, vida_util, tasa_descuento, eficiencia_termica, eficiencia_optica, factor_perdidas):
    capacidad = area_colectora / 10
    produccion_neta = produccion_anual * eficiencia_termica * eficiencia_optica * (1 - factor_perdidas)
    costo_total = capacidad * capex
    opex_anual = costo_total * opex_pct
    fcr = (tasa_descuento * (1 + tasa_descuento) ** vida_util) / ((1 + tasa_descuento) ** vida_util - 1)
    costo_anual = costo_total * fcr + opex_anual
    lcoe = costo_anual / produccion_neta
    return lcoe, produccion_neta

# === 4. Generar resultados combinados (PV + CSP) ===
resultados_pv = []
for _, fila in df_pv.iterrows():
    capacidad = fila['Capacidad (kW)']
    produccion_base = fila['Producción Anual (kWh)']
    for r, opex, eta_inv, capex, vida, deg, loss in itertools.product(
        tasas_descuento, opex_porcentajes_pv, eficiencias_inversor,
        capex_values_pv, vidas_utiles, degradaciones, perdidas_sistema):
        lcoe, prod_neta = calcular_lcoe_pv(capacidad, produccion_base, capex, opex, vida, r, deg, loss, eta_inv)
        resultados_pv.append({
            'País': fila['País'], 'Tecnología': 'PV', 'Capacidad (kW)': capacidad,
            'Producción Anual (kWh)': produccion_base, 'Producción Neta (kWh)': prod_neta,
            'Tasa Descuento': r, 'OPEX (%)': opex, 'CAPEX ($/kW)': capex,
            'Vida Útil (años)': vida, 'LCOE ($/kWh)': lcoe
        })

resultados_csp = []
for _, fila in df_csp.iterrows():
    area_colectora = fila['Área Colectora (m²)']
    produccion_base = fila['Producción Anual (MWh)'] * 1000
    for r, opex, eta_term, capex, vida, eta_opt, loss in itertools.product(
        tasas_descuento, opex_porcentajes_csp, eficiencias_termicas,
        capex_values_csp, vidas_utiles, eficiencias_opticas, factores_perdidas):
        lcoe, prod_neta = calcular_lcoe_csp(area_colectora, produccion_base, capex, opex, vida, r, eta_term, eta_opt, loss)
        resultados_csp.append({
            'País': fila['País'], 'Tecnología': 'CSP', 'Capacidad (kW)': area_colectora / 10,
            'Área Colectora (m²)': area_colectora, 'Producción Anual (kWh)': produccion_base,
            'Producción Neta (kWh)': prod_neta, 'Tasa Descuento': r,
            'OPEX (%)': opex, 'CAPEX ($/kW)': capex, 'Vida Útil (años)': vida,
            'LCOE ($/kWh)': lcoe
        })

df_lcoe_pv = pd.DataFrame(resultados_pv)
df_lcoe_csp = pd.DataFrame(resultados_csp)
df_lcoe_combinado = pd.concat([df_lcoe_pv, df_lcoe_csp], ignore_index=True)
df_lcoe_combinado.to_csv('lcoe_sensibilidad_combinado.csv', index=False)

# === 5. Análisis de sensibilidad por país y tecnología ===
tasa_descuento_base = 0.07
vida_util_base = 25
degradacion_base = 0.005
rango_tasas = np.linspace(0.03, 0.12, 8)
rango_capex = np.linspace(0.7, 1.3, 8)
rango_opex = np.linspace(0.7, 1.3, 8)
rango_degrad = np.linspace(0, 0.02, 8)
rango_rend = np.linspace(0.94, 1.0, 8)
rango_vida = np.arange(15, 36, 3)

def calcular_lcoe_simple(capex, opex_anual, produccion_anual, vida_util, tasa_descuento, degradacion):
    numerador = capex + sum([opex_anual / ((1 + tasa_descuento) ** t) for t in range(1, vida_util + 1)])
    denominador = sum([produccion_anual * ((1 - degradacion) ** (t-1)) / ((1 + tasa_descuento) ** t) for t in range(1, vida_util + 1)])
    return numerador / denominador

def sensibilidad_por_pais(df, tecnologia):
    for pais in df['País'].unique():
        nombre_archivo = f'sensibilidad_lcoe_{tecnologia}_{pais}.png'
        if os.path.exists(nombre_archivo):
            print(f'🟡 Ya existe: {nombre_archivo}')
            continue

        df_pais = df[df['País'] == pais]
        if tecnologia == 'PV':
            capex = df_pais['Capacidad (kW)'].mean() * 800
            opex = 0.015 * capex
            produccion = df_pais.iloc[0]['Producción Anual (kWh)']
        else:
            area_colectora = df_pais['Área Colectora (m²)'].mean()
            capex = (area_colectora / 10) * 4000
            opex = 0.015 * capex
            produccion = df_pais.iloc[0]['Producción Anual (MWh)'] * 1000

        lcoe_tasas = [calcular_lcoe_simple(capex, opex, produccion, vida_util_base, t, degradacion_base) for t in rango_tasas]
        lcoe_capex = [calcular_lcoe_simple(capex*m, opex, produccion, vida_util_base, tasa_descuento_base, degradacion_base) for m in rango_capex]
        lcoe_opex = [calcular_lcoe_simple(capex, opex*m, produccion, vida_util_base, tasa_descuento_base, degradacion_base) for m in rango_opex]
        lcoe_deg = [calcular_lcoe_simple(capex, opex, produccion, vida_util_base, tasa_descuento_base, d) for d in rango_degrad]
        lcoe_rend = [calcular_lcoe_simple(capex, opex, produccion*r, vida_util_base, tasa_descuento_base, degradacion_base) for r in rango_rend]
        lcoe_vida = [calcular_lcoe_simple(capex, opex, produccion, v, tasa_descuento_base, degradacion_base) for v in rango_vida]

        # Gráfico
        plt.figure(figsize=(14,8))
        plt.suptitle(f'Sensibilidad LCOE - {tecnologia} - {pais}', fontsize=16)
        plt.subplot(231)
        plt.plot(rango_tasas, lcoe_tasas, marker='o'); plt.xlabel('Tasa de descuento'); plt.ylabel('LCOE (USD/kWh)'); plt.title('FCR')
        plt.subplot(232)
        plt.plot(rango_capex, lcoe_capex, marker='o'); plt.xlabel('Multiplicador CAPEX'); plt.title('CAPEX')
        plt.subplot(233)
        plt.plot(rango_opex, lcoe_opex, marker='o'); plt.xlabel('Multiplicador OPEX'); plt.title('OPEX')
        plt.subplot(234)
        plt.plot(rango_degrad*100, lcoe_deg, marker='o'); plt.xlabel('Degradación anual (%)'); plt.title('Degradación')
        plt.subplot(235)
        plt.plot(rango_rend*100, lcoe_rend, marker='o'); plt.xlabel('Rendimiento inversor (%)'); plt.title('Rendimiento inversor')
        plt.subplot(236)
        plt.plot(rango_vida, lcoe_vida, marker='o'); plt.xlabel('Vida útil (años)'); plt.title('Vida útil')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(nombre_archivo)
        plt.close()
        print(f'✅ Guardado: {nombre_archivo}')

# === Ejecutar análisis ===
sensibilidad_por_pais(df_pv, 'PV')
sensibilidad_por_pais(df_csp, 'CSP')

print('✅ ¡Análisis de sensibilidad completado solo con gráficos detallados!')


🟡 Ya existe: sensibilidad_lcoe_PV_Chile.png
🟡 Ya existe: sensibilidad_lcoe_PV_China.png
🟡 Ya existe: sensibilidad_lcoe_PV_Rusia.png
🟡 Ya existe: sensibilidad_lcoe_CSP_Chile.png
🟡 Ya existe: sensibilidad_lcoe_CSP_China.png
🟡 Ya existe: sensibilidad_lcoe_CSP_Rusia.png
✅ ¡Análisis de sensibilidad completado solo con gráficos detallados!
